In [1]:
import os
import numpy as np
import torch
import pickle
from tqdm import tqdm
import scipy

In [4]:
# convert Diego's predictions to numpy, rearange the dimensions
def prepare_npy_files(path_ensamble):
    name_list = os.listdir(path_ensamble)
    for name in name_list:
        for index in tqdm(range(30), leave=False):
            file_path_input = os.path.join(path_ensamble, name, f'{index}.pkl')
            with open(file_path_input, 'rb') as pickle_file:
                try:
                    content = pickle.load(pickle_file)
                    if name == 'Diego':
                        content = content.squeeze(0).permute(0, 2, 3, 1).detach().cpu().numpy()
                    elif name == 'Traudi-Beatrice':
                        content = content.cpu().detach().numpy()
                    elif name == 'Kate':
                        content = content.squeeze(0).cpu().detach().numpy()
                    np.save(os.path.join(path_ensamble, name, f'{index}.npy'), content)
                except Exception as e:
                    print(f'({name}) Exception at: {index}: {e}')

In [5]:
path_ensamble = os.path.join('.', 'Ensamble')
prepare_npy_files(path_ensamble)

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:03<00:00,  7.24it/s]

(Piyalitt) Exception at: 29: Ran out of input


In [9]:
names_list = os.listdir(path_ensamble)
num_inference = 29 # not 30 because the last file of Piyalitt can not be read, thows exception

path_avg_ensamble = os.path.join('.', 'Ensamble_avg')
os.makedirs(path_avg_ensamble, exist_ok=True)

for index_filename in tqdm(range(num_inference), leave=False):
    # placeholder to hold the data of each person of the current index
    data_npy_list = []
    
    # read the current index file of each person and append to the list
    for index_name, name in enumerate(names_list):
        file_path = os.path.join(path_ensamble, name, f'{index_filename}.npy')
        
        data_npy = np.load(file_path)
        data_npy = torch.softmax(torch.tensor(data_npy), dim=0).detach().cpu().numpy()
        data_npy_list.append(data_npy)
        
    # convert to array and calculate the mean
    data_npy_list = np.array(data_npy_list)
    data_npy_mean = np.mean(data_npy_list, axis=0)

    # TODO calculate dice/mse/ce here
    
    # save the average of the ensamble
    path_out_current = os.path.join(path_avg_ensamble, f'{index_filename}.npy')
    np.save(path_out_current, data_npy_mean)

['Diego', 'Kate', 'Piyalitt', 'Traudi-Beatrice']